<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-Случайный-лес" data-toc-modified-id="Модель-Случайный-лес-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель Случайный лес</a></span></li><li><span><a href="#Модель-LGBM" data-toc-modified-id="Модель-LGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель LGBM</a></span></li><li><span><a href="#Модель-Логистическая-регрессия" data-toc-modified-id="Модель-Логистическая-регрессия-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

# Проект по классификации комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [2]:
state = 12345

In [3]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df_toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
df_toxic_comments.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


In [6]:
df_toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [7]:
def get_wordnet_pos(word):  # функция для лемматизации с POS-тегом
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [8]:
def lemmatize(text):   # функция лемматизации
    
    wnl = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    
    lemmatized_output = ' '.join([wnl.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    
    return lemmatized_output

In [9]:
def clear_text(text):   # функция регуляризации
    
    sub = re.sub(r'[^a-zA-Z]', ' ', text)  
    
    return ' '.join(sub.split())

Лемматизируем и очистим исходный текст:

In [10]:
df_toxic_comments['lem_text'] = df_toxic_comments['text'].apply(clear_text)

In [11]:
df_toxic_comments['lem_text'] = df_toxic_comments['lem_text'].apply(lemmatize)

In [12]:
df_toxic_comments['lem_text'] = df_toxic_comments['lem_text'].apply(lambda x: x.lower())

In [13]:
df_toxic_comments.head(3)

,text,toxic,lem_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...


Разделим данные на выборки в соотношении 3:1 и выделим целевой признак:

In [14]:
toxic_train, toxic_test = train_test_split(df_toxic_comments, test_size=0.25, random_state=state)

In [15]:
target_train = toxic_train['toxic']
target_test = toxic_test['toxic']

Создадим корпусы:

In [16]:
corpus = toxic_train['lem_text'].values
corpus_test = toxic_test['lem_text'].values

Посчитаем TF-IDF:

In [17]:
stop_words = set(stopwords.words('english'))

In [18]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

tf_idf = count_tf_idf.fit_transform(corpus)
tf_idf_test = count_tf_idf.transform(corpus_test)

print("Размер матрицы:", tf_idf.shape) 
print("Размер матрицы:", tf_idf_test.shape) 

Размер матрицы: (119678, 135088)
Размер матрицы: (39893, 135088)


Выводы о проделанной работе:

- импортировали библиотеки
- прочли данные 
- выполнили лемматизацию и очистку текстовых данных
- разделили данные на выборки
- посчитали TF-IDF для корпусов текстов

## Обучение

Обучим разные модели и подберем гиперпараметры:

In [19]:
scorer = make_scorer(f1_score, average='weighted', greater_is_better=True)

### Модель Случайный лес

In [20]:
rfc = RandomForestClassifier(random_state=state)

parametrs = {'n_estimators': range (1, 11, 2),
              'max_depth': range (1, 11, 2)}

In [21]:
grid = GridSearchCV(rfc, parametrs, cv=3, scoring=scorer)
grid.fit(tf_idf[:5000], target_train[:5000])

grid.best_score_ 

0.8549344830978797

In [22]:
grid.best_params_

{'max_depth': 7, 'n_estimators': 1}

### Модель LGBM

In [23]:
lgc = LGBMClassifier(random_state=state)

parametrs = {'n_estimators': range (1, 11, 3),
              'max_depth': range (1, 11, 3)}

In [24]:
grid = GridSearchCV(lgc, parametrs, cv=3, scoring=scorer)
grid.fit(tf_idf[:1000], target_train[:1000])

grid.best_score_ 

0.854077573321684

In [25]:
grid.best_params_

{'max_depth': 1, 'n_estimators': 1}

### Модель Логистическая регрессия

In [38]:
lr = LogisticRegression(random_state=state, solver='liblinear')

parametrs = {'penalty':['l1','l2'], 
              'C':[1, 10, 100]}

In [39]:
grid = GridSearchCV(lr, parametrs, cv=3, scoring=scorer)
grid.fit(tf_idf[:5000], target_train[:5000])

grid.best_score_ 

0.9398665453276186

In [40]:
grid.best_params_

{'C': 10, 'penalty': 'l1'}

Выводы:

- подобрали гиперпараметры для моделей Случайный лес, LGBM и Логистическая регрессия
 - для модели `RandomForestClassifier` F1 равен 0.855
 - для модели `LGBMClassifier` F1 равен 0.854
  - для модели `LogisticRegression` F1 равен 0.91
 
Выбираем модель с наилучшим результатом - `LogisticRegression`.

## Тестирование

In [41]:
model = LogisticRegression(class_weight="balanced", penalty='l1', 
                           C=10, solver='liblinear', random_state=state)

model.fit(tf_idf, target_train)

LogisticRegression(C=10, class_weight='balanced', penalty='l1',
                   random_state=12345, solver='liblinear')

In [42]:
predicted_test = model.predict(tf_idf_test)

In [43]:
print("f1:", f1_score(target_test, predicted_test))

f1: 0.7521703900914457


## Выводы

Значение метрики F1 на тестовой выборке для модели Логистическая регрессия равно 0.752, что удовлетворяет условию задания.